# 0: Import packages

In [110]:
from PIL import Image, ImageFile #pip install Pillow==9.4.0
import sys
import os
import numpy as np
import logging
import glob
import subprocess

logging.basicConfig(level=logging.INFO)
Image.MAX_IMAGE_PIXELS=None
ImageFile.LOAD_TRUNCATED_IMAGES = True

# 1: Specify filepaths

In [111]:
# Name folders for raw data and processed data
map_data_topfolder = 'raw_maps_20231024'
map_crops_topfolder = 'processed/crops'
map_streg_topfolder = 'processed/streg'

for fp in [map_crops_topfolder, map_streg_topfolder]:
    if not os.path.isdir(fp):
        os.makedirs(fp)

# IMPORTANT! Locate spotter directory and detectron weights
spotter_directory = 'C:/Users/HP/mapkurator-spotter-main/spotter-v2'
model_weights = 'C:/Users/HP/detectron2-main/detectron2/checkpoint/model_v2_en.pth'

# For assistance, see how the data should be organized
from IPython.display import Image as ipyim
#ipyim(filename="./directory_organization.jpg", width=400, height = 200)

# 2: Crop all jpeg maps in (user defined) map_data_topfolder

In [112]:
def crop(img_path, output_dir):

    map_name = os.path.basename(img_path).split('.')[0] # get the map name without extension
    output_dir = os.path.join(output_dir, map_name)

    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)

    map_img = Image.open(img_path) 
    width, height = map_img.size 

    #print(width, height)

    shift_size = 1000

    # pad the image to the size divisible by shift-size
    num_tiles_w = int(np.ceil(1. * width / shift_size))
    num_tiles_h = int(np.ceil(1. * height / shift_size))
    enlarged_width = int(shift_size * num_tiles_w)
    enlarged_height = int(shift_size * num_tiles_h)

    enlarged_map = Image.new(mode="RGB", size=(enlarged_width, enlarged_height))
    # paste map_imge to enlarged_map
    enlarged_map.paste(map_img) 

    for idx in range(0, num_tiles_h):
        for jdx in range(0, num_tiles_w):
            img_clip = enlarged_map.crop((jdx * shift_size, idx * shift_size,(jdx + 1) * shift_size, (idx + 1) * shift_size, ))

            out_path = os.path.join(output_dir, 'h' + str(idx) + '_w' + str(jdx) + '.jpg')
            img_clip.save(out_path)

    logging.info('Done cropping %s' %img_path )

# Run crop on all maps
for map_data_subfolder in next(os.walk(map_data_topfolder))[1]:
    jpeg_list = glob.glob(map_data_topfolder + '/' + map_data_subfolder + '/*.jpeg')
    if len(jpeg_list) != 1:
        print(map_data_subfolder + " failed. Please ensure there is exactly 1 file with extension .jpeg in the folder.")
    else:
        map_image = jpeg_list[0].split("\\")[1]
        crop(map_data_topfolder + '/' + map_data_subfolder + "/" + map_image, map_crops_topfolder)

# ex. hardcoded crop on one map
# crop('original_maps/1858_pierotti/pierotti_1858.jpeg', 'cropped_maps')


1818_sieber failed. Please ensure there is exactly 1 file with extension .jpeg in the folder.
1822_richardson failed. Please ensure there is exactly 1 file with extension .jpeg in the folder.
1823_westphal failed. Please ensure there is exactly 1 file with extension .jpeg in the folder.


INFO:root:Done cropping raw_maps_20231024/1827_failoni/failoni_1827.jpeg
INFO:root:Done cropping raw_maps_20231024/1835_catherwood/catherwood_1835.jpeg
INFO:root:Done cropping raw_maps_20231024/1835_catherwood_2/catherwood_1835_2.jpeg


1837_dufour failed. Please ensure there is exactly 1 file with extension .jpeg in the folder.


INFO:root:Done cropping raw_maps_20231024/1837_niebuhr/niebuhr_1837.jpeg
INFO:root:Done cropping raw_maps_20231024/1841_aldrich_symonds/aldrich_symonds_1841.jpeg
INFO:root:Done cropping raw_maps_20231024/1841_kiepert/kiepert_1841.jpeg
INFO:root:Done cropping raw_maps_20231024/1841_kiepert_2/kiepert_1841_2.jpeg
INFO:root:Done cropping raw_maps_20231024/1843_ball/ball_1843.jpeg
INFO:root:Done cropping raw_maps_20231024/1845_kiepert/kiepert_1845.jpeg


1846_krafft failed. Please ensure there is exactly 1 file with extension .jpeg in the folder.


...

1883_sandreczki failed. Please ensure there is exactly 1 file with extension .jpeg in the folder.


INFO:root:Done cropping raw_maps_20231024/1890_thuillier/thuillier_1890.jpeg


1894_schick failed. Please ensure there is exactly 1 file with extension .jpeg in the folder.


INFO:root:Done cropping raw_maps_20231024/1895_riess/riess_1895.jpeg


1911_vincent failed. Please ensure there is exactly 1 file with extension .jpeg in the folder.


# 3: Scan for text in all crops in (user defined) map_scan_topfolder

In [114]:
# Set up spotter arguments
gpu_id = 0
spotter_model = 'spotter-v2'
spotter_config = spotter_directory + '/configs/PALEJUN/Finetune/Base-SynthMap-Polygon.yaml'
inference_py_loc = spotter_directory + '/tools/inference.py'
curr_wd = os.getcwd().replace("\\","/")

# Run spotter on all files in processed/crops/
command = f'python {inference_py_loc} --config-file {spotter_config} --model_weights {model_weights} --output_json --inputs True --curr_wd {curr_wd} --map_crops_topfolder {map_crops_topfolder} --map_streg_topfolder {map_streg_topfolder}'
subprocess.run(command) #, shell=True)

# if you remove --output_json, it will output a picture of the image with recognized text overlaid

# here is a sample command if you want to run on one image (you must define filepath_to_jpg and filepath). note --output_json is removed (see comment above).
command = f'python {inference_py_loc} --config-file {spotter_config} --model_weights {model_weights} --input {filepath_to_jpg} --map_streg_topfolder {filepath}'

CompletedProcess(args='python C:/Users/HP/mapkurator-spotter-main/spotter-v2/tools/inference.py --config-file C:/Users/HP/mapkurator-spotter-main/spotter-v2/configs/PALEJUN/Finetune/Base-SynthMap-Polygon.yaml --model_weights C:/Users/HP/detectron2-main/detectron2/checkpoint/model_v2_en.pth --output_json --inputs True --curr_wd c:/Users/HP/Documents/GitHub/jerusalem_maps_epfl_dh405 --map_crops_topfolder processed/crops --map_streg_topfolder processed/streg', returncode=0)